In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#pip list
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [1]:
!pip install -qq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.

In [ ]:
dataset=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_2_train.csv")
X=list(dataset['text'])
y=list(dataset['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=42,test_size=0.3)
train_df = pd.DataFrame({"text": X_train, "label": y_train})
test_df = pd.DataFrame({"text": X_test, "label": y_test})

In [5]:
dataset1=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_1_train.csv")
dataset1_filtered = dataset1[dataset1['label'] != 2]
X_train=list(dataset1_filtered['text'])
y_train=list(dataset1_filtered['label'])
train_df = pd.DataFrame({"text": X_train, "label": y_train})
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_1_test.csv")
dataset2_filtered = dataset2[dataset2['label'] != 2]
X_test=list(dataset2_filtered['text'])
y_test=list(dataset2_filtered['label'])
test_df = pd.DataFrame({"text": X_test, "label": y_test})

In [6]:
train_df.shape

(33685, 2)

In [7]:
test_df.shape

(4216, 2)

In [8]:
import torch
torch.cuda.device_count()

1

In [9]:
device = torch.cuda.get_device_properties(0)  # Assuming you have at least one GPU
print(device)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [ ]:
#os.makedirs('/content/sample_data/')

In [10]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [11]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [12]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [16]:
import pickle
xlm_roberta_tamil="/content/drive/MyDrive/Capstone/Models/Tamil/xlm_roberta_tamil.sav"
distilbert_tamil="/content/drive/MyDrive/Capstone/Models/Tamil/distilbert_tamil.sav"
muril_tamil="/content/drive/MyDrive/Capstone/Models/Tamil/muril_tamil.sav"
mbert_tamil="/content/drive/MyDrive/Capstone/Models/Tamil/mbert_tamil.sav"
tamillion_tamil="/content/drive/MyDrive/Capstone/Models/Tamil/tamillion_tamil.sav"
xlm_model_tamil=pickle.load(open(xlm_roberta_tamil,'rb'))
mbert_model_tamil=pickle.load(open(mbert_tamil,'rb'))
distilbert_model_tamil=pickle.load(open(distilbert_tamil,'rb'))
muril_model_tamil=pickle.load(open(muril_tamil,'rb'))
tamillion_model_tamil=pickle.load(open(tamillion_tamil,'rb'))

# XLM_RoBERTa

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 4,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 1e-5,
    'train_batch_size': 64,
    "use_early_stopping": True,
}

xlm_model_tamil = ClassificationModel(
    "xlmroberta",
    "xlm-roberta-base",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:484: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [ ]:
xlm_model_tamil.train_model(train_df)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 4 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

(9220, 0.23025401422234734)

In [17]:
predictions, raw_outputs = xlm_model_tamil.predict(list(test_df['text'].values))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

In [18]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['label'].values,zero_division=True))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      3186
           1       0.88      0.87      0.88      1030

    accuracy                           0.94      4216
   macro avg       0.92      0.92      0.92      4216
weighted avg       0.94      0.94      0.94      4216



In [19]:
accuracy = accuracy_score(test_df['label'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9395161290322581


In [20]:
f1 = f1_score(test_df['label'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.9395860572703891


In [ ]:
import pickle
# save the model to disk
filename = 'xlm_roberta_tamil.sav'
pickle.dump(xlm_model_tamil, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import shutil

local_model_path = '/content/sample_data/xlm_roberta_tamil.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Tamil/xlm_roberta_tamil.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Tamil/xlm_roberta_tamil.sav'

# mBERT

In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 2,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 2e-5,
    'train_batch_size': 64,
    "gradient_accumulation_steps":2,
    "use_early_stopping": True,
}

mbert_model_tamil = ClassificationModel(
    "bert",
    "bert-base-multilingual-uncased",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
mbert_model_tamil.train_model(train_df)

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

(4600, 0.13078983397185567)

In [21]:
predictions, raw_outputs = mbert_model_tamil.predict(list(test_df['text'].values))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

In [22]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['label'].values,zero_division=True))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      3282
           1       0.84      0.92      0.88       934

    accuracy                           0.94      4216
   macro avg       0.91      0.94      0.92      4216
weighted avg       0.95      0.94      0.95      4216



In [23]:
accuracy = accuracy_score(test_df['label'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9442599620493358


In [24]:
f1 = f1_score(test_df['label'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.9433755739310461


In [ ]:
import pickle
# save the model to disk
filename = 'mbert_tamil.sav'
pickle.dump(mbert_model_tamil, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
sample_text="நெறைய நெறைய நெறைய like views vanthukitte irukku"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0])

In [ ]:
sample_text="GVM tha erangi senji irukan"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
import shutil

local_model_path = '/content/sample_data/mbert_tamil.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Tamil/mbert_tamil.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Tamil/mbert_tamil.sav'

# DistilBERT



In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 2,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 2e-5,
    'train_batch_size': 64,
    "gradient_accumulation_steps":2,
    "use_early_stopping": True,
}

distilbert_model_tamil = ClassificationModel(
    "bert",
    "distilbert-base-multilingual-cased",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.ou

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [ ]:
distilbert_model_tamil.train_model(train_df)

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

(4600, 0.2920235427699821)

In [25]:
predictions, raw_outputs = distilbert_model_tamil.predict(list(test_df['text'].values))
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['label'].values,zero_division=True))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3241
           1       0.77      0.81      0.79       975

    accuracy                           0.90      4216
   macro avg       0.86      0.87      0.86      4216
weighted avg       0.90      0.90      0.90      4216



In [26]:
accuracy = accuracy_score(test_df['label'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9003795066413662


In [27]:
f1 = f1_score(test_df['label'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.899554466789194


In [ ]:
import pickle
# save the model to disk
filename = 'distilbert_tamil.sav'
pickle.dump(distilbert_model_tamil, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import shutil

local_model_path = '/content/sample_data/distilbert_tamil.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Tamil/distilbert_tamil.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Tamil/distilbert_tamil.sav'

# Muril

In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 2,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 2e-5,
    'train_batch_size': 64,
    "gradient_accumulation_steps":2,
    "use_early_stopping": True,
}

muril_model_tamil = ClassificationModel(
    "bert",
    "google/muril-base-cased",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
muril_model_tamil.train_model(train_df)

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

(4600, 0.22592315199005458)

In [28]:
predictions, raw_outputs = muril_model_tamil.predict(list(test_df['text'].values))
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['label'].values,zero_division=True))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      3233
           1       0.86      0.90      0.88       983

    accuracy                           0.94      4216
   macro avg       0.92      0.93      0.92      4216
weighted avg       0.94      0.94      0.94      4216



In [29]:
accuracy = accuracy_score(test_df['label'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9435483870967742


In [30]:
f1 = f1_score(test_df['label'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.9431612569698311


In [ ]:
import pickle
# save the model to disk
filename = 'muril_tamil.sav'
pickle.dump(muril_model_tamil, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import shutil

local_model_path = '/content/sample_data/muril_tamil.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Tamil/muril_tamil.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Tamil/muril_tamil.sav'

# Tamillion

In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 5,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 2.5e-5,
    'train_batch_size': 64,
    "gradient_accumulation_steps":2,
    "use_early_stopping": True,
}

tamillion_model_tamil = ClassificationModel(
    "bert",
    "monsoon-nlp/tamillion",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monsoon-nlp/tamillion and are newly initialized: ['encoder.layer.11.intermediate.dense.weight', 'encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.9.output.dense.bias', 'encoder.layer.11.output.dense.bias', 'encoder.layer.3.attention.output.LayerNorm.weight', 'encoder.layer.5.attention.output.dense.weight', 'encoder.layer.9.attention.self.query.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.7.intermediate.dense.bias', 'embeddings.LayerNorm.weight', 'encoder.layer.2.attention.self.query.weight', 'encoder.layer.3.attention.output.dense.weight', 'encoder.layer.1.attention.output.LayerNorm.bias', 'encoder.layer.5.output.LayerNorm.weight', 'encoder.layer.6.attention.self.value.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.11.attention.self.query.weight', 'encoder.layer.9.output.LayerNorm.bias', 'encoder.layer.7.attention.self.query.weight', 'encoder.lay

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/837k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [ ]:
tamillion_model_tamil.train_model(train_df)

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

(4600, 0.31376601392484227)

In [31]:
predictions, raw_outputs = tamillion_model_tamil.predict(list(test_df['text'].values))
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['label'].values,zero_division=True))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3271
           1       0.73      0.79      0.75       945

    accuracy                           0.89      4216
   macro avg       0.83      0.85      0.84      4216
weighted avg       0.89      0.89      0.89      4216



In [32]:
accuracy = accuracy_score(test_df['label'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8851992409867173


In [33]:
f1 = f1_score(test_df['label'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.8836168666633522


In [ ]:
import pickle
# save the model to disk
filename = 'tamillion_tamil.sav'
pickle.dump(tamillion_model_tamil, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import shutil

local_model_path = '/content/sample_data/tamillion_tamil.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Tamil/tamillion_tamil.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Tamil/tamillion_tamil.sav'